In [1]:
import asyncio
import json
import os
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Role,
    WorkflowBuilder,
    WorkflowContext,
    ai_function,
    executor,
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")

✅ All imports successful!


## Βήμα 1: Ορισμός Μοντέλων Pydantic για Δομημένα Αποτελέσματα

Αυτά τα μοντέλα ορίζουν το **σχήμα** που θα επιστρέφουν οι agents. Η χρήση του `response_format` με το Pydantic εξασφαλίζει:
- ✅ Ασφαλή εξαγωγή δεδομένων βάσει τύπου
- ✅ Αυτόματη επικύρωση
- ✅ Χωρίς σφάλματα ανάλυσης από απαντήσεις ελεύθερου κειμένου
- ✅ Εύκολη δρομολόγηση βάσει πεδίων


In [2]:
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""

    destination: str
    has_availability: bool
    message: str


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""

    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""

    destination: str
    action: str
    message: str


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")

✅ Pydantic models defined:
   - BookingCheckResult (availability check)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)


## Βήμα 2: Δημιουργία του Εργαλείου Κρατήσεων Ξενοδοχείου

Αυτό το εργαλείο είναι αυτό που θα καλεί ο **availability_agent** για να ελέγξει αν υπάρχουν διαθέσιμα δωμάτια. Χρησιμοποιούμε τον διακοσμητή `@ai_function` για να:
- Μετατρέψουμε μια συνάρτηση Python σε εργαλείο που μπορεί να καλείται από AI
- Δημιουργήσουμε αυτόματα JSON schema για το LLM
- Διαχειριστούμε την επικύρωση παραμέτρων
- Ενεργοποιήσουμε την αυτόματη κλήση από πράκτορες

Για αυτή την επίδειξη:
- **Στοκχόλμη, Σιάτλ, Τόκιο, Λονδίνο, Άμστερνταμ** → Υπάρχουν δωμάτια ✅
- **Όλες οι άλλες πόλεις** → Δεν υπάρχουν δωμάτια ❌


In [3]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## Βήμα 3: Ορισμός Συναρτήσεων Συνθηκών για Δρομολόγηση

Αυτές οι συναρτήσεις εξετάζουν την απάντηση του agent και καθορίζουν ποια διαδρομή θα ακολουθηθεί στη ροή εργασίας.

**Βασικό Μοτίβο:**
1. Ελέγξτε αν το μήνυμα είναι `AgentExecutorResponse`
2. Αναλύστε την δομημένη έξοδο (μοντέλο Pydantic)
3. Επιστρέψτε `True` ή `False` για να ελέγξετε τη δρομολόγηση

Η ροή εργασίας θα αξιολογήσει αυτές τις συνθήκες στις **ακμές** για να αποφασίσει ποιον εκτελεστή να καλέσει στη συνέχεια.


In [4]:
def has_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels ARE available.
    
    Returns True if the destination has hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return True  # Default to True if unexpected type

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}
            </div>
        """)
        )

        return result.has_availability
    except Exception as e:
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                <strong>⚠️  Error:</strong> {str(e)}
            </div>
        """)
        )
        return False


def no_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels are NOT available.
    
    Returns True if the destination has no hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )

        return not result.has_availability
    except Exception as e:
        return False


print("✅ Condition functions defined:")
print("   - has_availability_condition (routes when rooms exist)")
print("   - no_availability_condition (routes when no rooms)")

✅ Condition functions defined:
   - has_availability_condition (routes when rooms exist)
   - no_availability_condition (routes when no rooms)


## Βήμα 4: Δημιουργία Προσαρμοσμένου Εκτελεστή Εμφάνισης

Οι εκτελεστές είναι στοιχεία της ροής εργασίας που εκτελούν μετασχηματισμούς ή παρενέργειες. Χρησιμοποιούμε τον διακοσμητή `@executor` για να δημιουργήσουμε έναν προσαρμοσμένο εκτελεστή που εμφανίζει το τελικό αποτέλεσμα.

**Βασικές Έννοιες:**
- `@executor(id="...")` - Καταχωρεί μια συνάρτηση ως εκτελεστή ροής εργασίας
- `WorkflowContext[Never, str]` - Υποδείξεις τύπου για είσοδο/έξοδο
- `ctx.yield_output(...)` - Επιστρέφει το τελικό αποτέλεσμα της ροής εργασίας


In [5]:
@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """
    Display the final result as workflow output.
    
    This executor receives the final agent response and yields it as the workflow output.
    """
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ display_result executor created with @executor decorator")

✅ display_result executor created with @executor decorator


## Βήμα 5: Φόρτωση Μεταβλητών Περιβάλλοντος

Ρυθμίστε τον πελάτη LLM. Αυτό το παράδειγμα λειτουργεί με:
- **Μοντέλα GitHub** (Δωρεάν επίπεδο με GitHub token)
- **Azure OpenAI**
- **OpenAI**


In [6]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(base_url=os.environ.get(
    "GITHUB_ENDPOINT"), api_key=os.environ.get("GITHUB_TOKEN"), model_id="gpt-4o")

## Βήμα 6: Δημιουργία Πρακτόρων AI με Δομημένα Αποτελέσματα

Δημιουργούμε **τρεις εξειδικευμένους πράκτορες**, καθένας τυλιγμένος σε ένα `AgentExecutor`:

1. **availability_agent** - Ελέγχει τη διαθεσιμότητα ξενοδοχείων χρησιμοποιώντας το εργαλείο
2. **alternative_agent** - Προτείνει εναλλακτικές πόλεις (όταν δεν υπάρχουν δωμάτια)
3. **booking_agent** - Ενθαρρύνει την κράτηση (όταν υπάρχουν διαθέσιμα δωμάτια)

**Βασικά Χαρακτηριστικά:**
- `tools=[hotel_booking]` - Παρέχει το εργαλείο στον πράκτορα
- `response_format=PydanticModel` - Επιβάλλει δομημένη έξοδο JSON
- `AgentExecutor(..., id="...")` - Τυλίγει τον πράκτορα για χρήση στη ροή εργασίας


In [7]:
# Agent 1: Check availability with tool
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), and message (string). "
            "The message should summarize the availability status."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
    ),
    id="availability_agent",
)

# Agent 2: Suggest alternative (when no rooms)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 3: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created 3 Agents:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - Checks availability with hotel_booking tool</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
        </ul>
    </div>
""")
)

## Βήμα 7: Δημιουργία της Ροής Εργασίας με Συνθήκες στις Ακμές

Τώρα χρησιμοποιούμε το `WorkflowBuilder` για να κατασκευάσουμε το γράφημα με δρομολόγηση βάσει συνθηκών:

**Δομή Ροής Εργασίας:**
```
availability_agent (START)
        ↓
   Evaluate conditions
        ↙         ↘
[no_availability]  [has_availability]
        ↓              ↓
alternative_agent  booking_agent
        ↓              ↓
    display_result ←───┘
```

**Βασικές Μέθοδοι:**
- `.set_start_executor(...)` - Ορίζει το σημείο εισόδου
- `.add_edge(from, to, condition=...)` - Προσθέτει ακμή με συνθήκη
- `.build()` - Ολοκληρώνει τη ροή εργασίας


In [8]:
# Build the workflow with conditional routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    # NO AVAILABILITY PATH
    .add_edge(availability_agent, alternative_agent, condition=no_availability_condition)
    .add_edge(alternative_agent, display_result)
    # HAS AVAILABILITY PATH
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Conditional Routing:</strong><br>
            • If <strong>NO availability</strong> → alternative_agent → display_result<br>
            • If <strong>availability</strong> → booking_agent → display_result
        </p>
    </div>
""")
)

## Βήμα 8: Εκτέλεση Δοκιμαστικής Περίπτωσης 1 - Πόλη ΧΩΡΙΣ Διαθεσιμότητα (Παρίσι)

Ας δοκιμάσουμε τη διαδρομή **χωρίς διαθεσιμότητα** ζητώντας ξενοδοχεία στο Παρίσι (το οποίο δεν έχει δωμάτια στη δική μας προσομοίωση).


In [9]:
display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Paris (No Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → alternative_agent → display_result</p>
    </div>
""")
)

# Create request for Paris
request_paris = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run the workflow
events_paris = await workflow.run(request_paris)
outputs_paris = events_paris.get_outputs()

# Display results
if outputs_paris:
    result_paris = AlternativeResult.model_validate_json(outputs_paris[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; box-shadow: 0 4px 12px rgba(255,165,0,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 WORKFLOW RESULT (Paris)</h3>
            <div style='background: white; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Alternative Suggestion:</strong> 🏨 {result_paris.alternative_destination}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Reason:</strong> {result_paris.reason}</p>
            </div>
        </div>
    """)
    )

## Βήμα 9: Εκτέλεση Δοκιμαστικής Περίπτωσης 2 - Πόλη ΜΕ Διαθεσιμότητα (Στοκχόλμη)

Τώρα ας δοκιμάσουμε τη διαδρομή **διαθεσιμότητας** ζητώντας ξενοδοχεία στη Στοκχόλμη (η οποία έχει δωμάτια στη προσομοίωσή μας).


In [10]:
display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 2: Stockholm (Has Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → booking_agent → display_result</p>
    </div>
""")
)

# Create request for Stockholm
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], should_respond=True
)

# Run the workflow
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT (Stockholm)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
            </div>
        </div>
    """)
    )

## Βασικά Σημεία και Επόμενα Βήματα

### ✅ Τι Έχετε Μάθει:

1. **Μοτίβο WorkflowBuilder**
   - Χρησιμοποιήστε `.set_start_executor()` για να ορίσετε το σημείο εκκίνησης
   - Χρησιμοποιήστε `.add_edge(from, to, condition=...)` για δρομολόγηση με βάση συνθήκες
   - Καλέστε `.build()` για να ολοκληρώσετε τη ροή εργασίας

2. **Δρομολόγηση με Συνθήκες**
   - Οι συναρτήσεις συνθηκών εξετάζουν το `AgentExecutorResponse`
   - Αναλύστε δομημένα αποτελέσματα για να λάβετε αποφάσεις δρομολόγησης
   - Επιστρέψτε `True` για να ενεργοποιήσετε μια σύνδεση, `False` για να την παρακάμψετε

3. **Ενσωμάτωση Εργαλείων**
   - Χρησιμοποιήστε `@ai_function` για να μετατρέψετε συναρτήσεις Python σε εργαλεία AI
   - Οι πράκτορες καλούν αυτόματα τα εργαλεία όταν χρειάζεται
   - Τα εργαλεία επιστρέφουν JSON που οι πράκτορες μπορούν να αναλύσουν

4. **Δομημένα Αποτελέσματα**
   - Χρησιμοποιήστε μοντέλα Pydantic για ασφαλή εξαγωγή δεδομένων
   - Ορίστε `response_format=MyModel` κατά τη δημιουργία πρακτόρων
   - Αναλύστε απαντήσεις με `Model.model_validate_json()`

5. **Προσαρμοσμένοι Εκτελεστές**
   - Χρησιμοποιήστε `@executor(id="...")` για να δημιουργήσετε στοιχεία ροής εργασίας
   - Οι εκτελεστές μπορούν να μετασχηματίσουν δεδομένα ή να εκτελέσουν ενέργειες
   - Χρησιμοποιήστε `ctx.yield_output()` για να παράγετε αποτελέσματα ροής εργασίας

### 🚀 Εφαρμογές στον Πραγματικό Κόσμο:

- **Κράτηση Ταξιδιών**: Έλεγχος διαθεσιμότητας, προτάσεις εναλλακτικών, σύγκριση επιλογών
- **Εξυπηρέτηση Πελατών**: Δρομολόγηση με βάση τον τύπο προβλήματος, το συναίσθημα, την προτεραιότητα
- **Ηλεκτρονικό Εμπόριο**: Έλεγχος αποθεμάτων, προτάσεις εναλλακτικών, επεξεργασία παραγγελιών
- **Εποπτεία Περιεχομένου**: Δρομολόγηση με βάση βαθμολογίες τοξικότητας, αναφορές χρηστών
- **Ροές Εγκρίσεων**: Δρομολόγηση με βάση το ποσό, τον ρόλο χρήστη, το επίπεδο κινδύνου
- **Επεξεργασία Πολλαπλών Σταδίων**: Δρομολόγηση με βάση την ποιότητα δεδομένων, την πληρότητα

### 📚 Επόμενα Βήματα:

- Προσθέστε πιο σύνθετες συνθήκες (πολλαπλά κριτήρια)
- Υλοποιήστε βρόχους με διαχείριση κατάστασης ροής εργασίας
- Προσθέστε υπο-ροές για επαναχρησιμοποιούμενα στοιχεία
- Ενσωματώστε με πραγματικά APIs (κρατήσεις ξενοδοχείων, συστήματα αποθεμάτων)
- Προσθέστε διαχείριση σφαλμάτων και εναλλακτικές διαδρομές
- Οπτικοποιήστε τις ροές εργασίας με τα ενσωματωμένα εργαλεία οπτικοποίησης



---

**Αποποίηση ευθύνης**:  
Αυτό το έγγραφο έχει μεταφραστεί χρησιμοποιώντας την υπηρεσία αυτόματης μετάφρασης [Co-op Translator](https://github.com/Azure/co-op-translator). Παρόλο που καταβάλλουμε προσπάθειες για ακρίβεια, παρακαλούμε να έχετε υπόψη ότι οι αυτοματοποιημένες μεταφράσεις ενδέχεται να περιέχουν λάθη ή ανακρίβειες. Το πρωτότυπο έγγραφο στη μητρική του γλώσσα θα πρέπει να θεωρείται η αυθεντική πηγή. Για κρίσιμες πληροφορίες, συνιστάται επαγγελματική ανθρώπινη μετάφραση. Δεν φέρουμε ευθύνη για τυχόν παρεξηγήσεις ή εσφαλμένες ερμηνείες που προκύπτουν από τη χρήση αυτής της μετάφρασης.
